# Import DATA

In [1]:
import os
import pandas as pd

os.chdir('../siksin')
from restaurant_url import sikshin_restaurant_url_src as siksin
from restaurant_review import siksin_review_list

ss_data = pd.DataFrame(siksin, columns=["name", "url", "img_url", "total_rating"])
ss_review = pd.DataFrame(siksin_review_list, columns=["platform", "name", "rating", "comment"]).drop_duplicates()

os.chdir('../kakao')
from kakao_restaurant import kakao_list as kakao
from kakao_restaurant_review import kakao_review_list as kakao_review

k_data = pd.DataFrame(kakao, columns=["name", "category", "total_rating", "rating_count", "img_url"])
k_review = pd.DataFrame(kakao_review, columns=["platform", "name", "rating", "rating_date", "comment"]).drop_duplicates()

os.chdir('../mangoplate')

m_data = pd.read_csv("mango_restaurant.csv", header=0, names=["name", "total_rating", "category", "img_url"])
m_review = pd.read_csv("mangoplate_review.csv", header=0, names=["name", "rating_date", "comment", "rating"]).drop_duplicates()

os.chdir('../naver')

n_data = pd.read_csv("naver_restaurant.csv", header=0, names=["name", "category", "total_rating"], usecols=[0, 1, 2])
n_review = pd.read_csv("naver_review.csv", header=0, names=["name", "rating_date", "comment", "rating"])
n_review = n_review.append(pd.read_csv("naver_review_by_tab.csv", header=0, names=["name", "rating_date", "comment", "rating"], sep="\t"), ignore_index=True).drop_duplicates()

os.chdir("../data_analysis")

ss_data["platform"] = "siksin"
n_data["platform"] = "naver"
k_data["platform"] = "kakao"
m_data["platform"] = "mango"

display(k_review.info(), ss_review.info(), m_review.info(), n_review.info(), ss_data.head(1), k_data.head(1), m_data.head(1), n_data.head(1), n_review.head(1))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7214 entries, 0 to 7219
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   platform     7214 non-null   object
 1   name         7214 non-null   object
 2   rating       7214 non-null   int64 
 3   rating_date  7214 non-null   object
 4   comment      7214 non-null   object
dtypes: int64(1), object(4)
memory usage: 338.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1963 entries, 0 to 2041
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   platform  1963 non-null   object
 1   name      1963 non-null   object
 2   rating    1963 non-null   object
 3   comment   1963 non-null   object
dtypes: object(4)
memory usage: 76.7+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2917 entries, 0 to 2922
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       -------------- 

None

None

None

None

,name,url,img_url,total_rating,platform
0,하남돼지집 선릉역삼점,https://www.siksinhot.com/P/369301,https://img.siksinhot.com/place/14639799276066...,4.3,siksin


,name,category,total_rating,rating_count,img_url,platform
0,뽕나무쟁이 선릉본점,"족발,보쌈",4.1,111,https://place.map.kakao.com/11190567,kakao


,name,total_rating,category,img_url,platform
0,모찌방,4.4,카페/디저트,https://mp-seoul-image-production-s3.mangoplat...,mango


,name,category,total_rating,platform
0,도쿄등심 선릉점,소고기구이,4.51,naver


,name,rating_date,comment,rating
0,명품유쾌 강남본점,2021.09.11,NaN,5.0


# 2개 이상의 플랫폼에서 리뷰를 가져온 음식점 찾기

In [2]:
display(k_review["name"].nunique(), ss_review["name"].nunique(), m_review["name"].nunique(), n_review["name"].nunique())

495

147

193

158

In [3]:
print(n_review["name"].nunique())

158


In [4]:
ss_names = set(ss_data["name"])
k_names = set(k_data["name"])
m_names = set(m_data["name"])
n_names = set(n_data["name"])

kn_names = k_names.union(n_names)
print(len(list(kn_names)))

knm_names = kn_names.union(m_names)
print(len(list(knm_names)))

all_names = knm_names.union(ss_names)
print(len(list(all_names)))

659
782
891


In [5]:
ss_np = ss_data[["name", "platform"]].set_index('name').rename(columns={"platform":"ss_platform"})
n_np = n_data[["name", "platform"]].set_index('name').rename(columns={"platform":"n_platform"})
k_np = k_data[["name", "platform"]].set_index('name').rename(columns={"platform":"k_platform"})
m_np = m_data[["name", "platform"]].set_index('name').rename(columns={"platform":"m_platform"})

In [6]:
kn_np = k_np.join(n_np)
knm_np = kn_np.join(m_np)
all_np = knm_np.join(ss_np)
all_np


,k_platform,n_platform,m_platform,ss_platform
name,,,,
1260이베리코흑돼지 강남직영점,kakao,NaN,NaN,NaN
24시와구아구,kakao,NaN,NaN,NaN
60년전통 신촌황소곱창 선릉직영점,kakao,NaN,NaN,NaN
7%칠백식당 포스코직영점,kakao,NaN,NaN,NaN
705사케집 선릉점,kakao,NaN,NaN,NaN
...,...,...,...,...
후웨이즈텐카민,kakao,NaN,NaN,NaN
후지,kakao,NaN,NaN,NaN
흑돈가 삼성역점,kakao,NaN,NaN,NaN


In [7]:
all_count = pd.DataFrame(4 - all_np.isnull().sum(axis = 1), columns=["count"])
all_count

,count
name,
1260이베리코흑돼지 강남직영점,1
24시와구아구,1
60년전통 신촌황소곱창 선릉직영점,1
7%칠백식당 포스코직영점,1
705사케집 선릉점,1
...,...
후웨이즈텐카민,1
후지,1
흑돈가 삼성역점,1


In [8]:
all_count.value_counts()

count
1        318
2        112
3         48
4         26
dtype: int64

In [9]:
review_only_one = all_count[all_count["count"] == 1]
name_only_one_list = set(review_only_one.index)
print(name_only_one_list, len(name_only_one_list))

{'이백 역삼점', '쇠뚜레', '젠제로', '껠끄쇼즈', '강남면옥 대치점', '카페베네 선릉역점', '해초록 별채사랑방', '스시류', '새집남원추어탕', '비블레스', '금수복국 대치점', '스타벅스 선릉로점', '24시와구아구', '미즈호노 스시야 대치점', '본죽 선릉역점', '일미집 강남직영점', '마구로젠', '배스킨라빈스 선릉점', '수림복국 본관', '이대성의진면목', '한앤둘치킨 선릉점', '김태주선산곱창 강남점', '카페봄', '오더에스프레소', '명인등심 삼성직영점', '화포식당 선릉점', '그란구스또', '딸부자네불백 역삼점', '싸프란겔', '엔제리너스 선릉골드로즈점', '빽다방 선릉중앙점', '롤리폴리꼬또', '파크루안 역삼점', '에그드랍 선릉역점', '츄라우미 역삼본점', '상아김밥 선릉직영점', '봉산집', '칠백식당 삼성직영점', '도쿄술집 선릉역점', '전가네손칼국수', '대파집', '클럽케이서울', '우밀면옥 상록점', '교대이층집 삼성점', '행복김밥', '황태마을', '빽다방 선릉역1호점', '우진가 더 챔버스', '써브웨이 삼성점', '지아니스나폴리 역삼점', '인기명 삼성점', '문배동육칼 삼성직영점', '투썸플레이스 대치선릉점', '부엉이식당', '바나프레소 선릉역사거리점', '대우부대찌개', '미성양꼬치 삼성점', '피양콩할마니 본점', '스타벅스 선릉역점', '우리집', '수담한정식', '신강양꼬치 선릉점', '소공순대', '붓처스컷 삼성점', '60년전통 신촌황소곱창 선릉직영점', '아오끼', '흑돈가 삼성역점', '브롱스 선릉점', '청춘화로 선릉점', '복돈이가마솥순대국', '에드빗커피', '로네펠트', '다노이 선릉점', '롯데리아 선릉점', '나향반점', '생활맥주 선릉역점', '마라공방 역삼점', '커피빈 선릉로86길점', '원조양평해장국', '마녀김밥 대치점', '식물원282', '신도세기 선릉점', '팀호완 삼성점', '이도곰탕 본점', '지구당 현대타워점', '피쉬스타 역삼점', '커피빈 선릉로

In [10]:
review_more_two = all_count[all_count["count"] != 1]
name_set = set(review_more_two.index)
# ss_name_list = list(ss_names.intersection(name_list))
# k_name_list = list(k_names.intersection(name_list))
# m_name_list = list(m_names.intersection(name_list))
print(name_set, len(name_set))

{'샹베리', '스타벅스 선릉세화빌딩점', '오징어풍경', '쏭타이 역삼점', '반포식스 선릉역점', '아야커피', '장진참치', '하얼빈가정식 본점', '황금어장', '하동관 강남분점', '재패니즈다이닝안심', '연타발 선릉점', '지유가오카핫초메 선릉점', '애니홀', '명랑회관 선릉점', '고반식당 선릉점', '달빛보쌈 논현점', '문어랑 상록점', '인평일등추어탕', '카츠젠 선릉점', '대파양곱창', '카발', '잭슨피자 대치점', '진풍정 강남점', '포하임 선릉점', '대도식당 삼성점', '청자골', '맛있는제주', '우마쿠라', '코벤트가든 선릉점', '국고집', '브라운돈까스 선릉점', '대치동집', '오카에리', '잼라운지', '도쿄등심 선릉점', '별미곳간', '슐미랭삼 선릉점', '투썸플레이스 테헤란선릉역점', '참치공간', '버드나무집 역삼점', '빌리엔젤 신라스테이점', '본죽&비빔밥cafe 선릉점', '조개사냥', '그릴1492 삼성1호점', '가연', '북창동순두부 선릉2호점', '동해수산', '양가식당', '하남돼지집 선릉1호점', '남원추어탕', '차호랑', '외고집설렁탕', '이승재참치', '고메홈', '잇쇼우', '제주은희네해장국 선릉직영점', '예향', '스시키', '커피마마퀸 선릉점', '라피자노스트라', '마라테이블', '미쓰양꼬치', '광주식당', '해피해피케이크', '마라차이즈', '전봇대', '평가옥 삼성점', '하루', '쌍룡갈비', '이디야커피 선릉역점', '선릉양조장족발', '어로장', '풍년집 선릉점', '샐러디 선릉점', '농민백암순대 본점', '모범떡볶이1976 선릉점', '코르바니', '연', '샤오바오우육면', '김봉남포장마차 선릉점', '동보성 강남점', '경천애인2237', '칠랑고', '청국장서갈비', '두레국수', '테라로사 포스코센터점', '수라선 역삼점', '삼성동커피볶는집', '몰타참치', '동래정 선릉직영점', '채린', '더피자스탠드', '큰입아구찜', '비언유주얼', '유하제과', 

In [11]:
review_more_two = all_count[all_count["count"] != 1]
name_list = list(review_more_two.index)
# ss_name_list = list(ss_names.intersection(name_list))
# k_name_list = list(k_names.intersection(name_list))
# m_name_list = list(m_names.intersection(name_list))
print(name_list, len(name_list))

['가나돈까스의집', '가연', '강남수랏간', '겐비이자카야', '경천애인2237', '고메홈', '고반식당 선릉점', '고운님한우곱창', '곰작골나주곰탕 선릉점', '광수사', '광주식당', '국고집', '군자대한곱창 선릉점', '그릴1492 삼성1호점', '기세끼 선릉점', '김돈이 본점', '김봉남포장마차 선릉점', '남원추어탕', '농민백암순대 본점', '니뽕내뽕 선릉점', '달빛보쌈 논현점', '대도식당 삼성점', '대치동집', '대파양곱창', '더피자스탠드', '도쿄등심 선릉점', '도쿄등심 선릉점', '돈그리아', '동래정 선릉직영점', '동보성 강남점', '동해수산', '두레국수', '등촌샤브칼국수 선릉점', '뚝심한우 선릉역점', '라무진 선릉역점', '라피자노스트라', '로브니', '마라차이즈', '마라테이블', '마초야', '맛있는제주', '맥도날드 선릉점', '메이크샐러드 선릉점', '명랑회관 선릉점', '모범떡볶이1976 선릉점', '모찌방', '몰타참치', '무한장어 역삼점', '문어랑 상록점', '문어랑 상록점', '미랑', '미랑', '미쓰양꼬치', '미트앤번버거샵 강남점', '바스버거 선릉점', '반룡산', '반포식스 선릉역점', '버드나무집 역삼점', '별미곳간', '별양집', '별양집', '보영만두 선릉점', '본죽&비빔밥cafe 선릉점', '부산양곱창', '북창동순두부 선릉2호점', '브라운돈까스 선릉점', '블랙랩', '비야게레로', '비언유주얼', '빌리앤오티스', '빌리엔젤 신라스테이점', '뽕나무쟁이 선릉본점', '삼미대포 선릉점', '삼성동커피볶는집', '샐러디 선릉점', '샤오바오우육면', '샹베리', '서래향', '선릉순대국', '선릉양조장족발', '세광양대창 선릉점', '세드라', '센도수산', '쇼유식당', '수다 대치점', '수라선 역삼점', '슐미랭삼 선릉점', '스노우폭스 선릉역점', '스시노스시', '스시마이우 선릉역점', '스시산원', '스시소라', '스시키', '스타벅스 선릉동신빌딩R점', '스타

In [12]:
name_list = sorted(list(name_set))

# 2개 이상의 플랫폼에서 리뷰가 존재하는 리뷰만 모으기

In [13]:
k_review = k_review[k_review["name"].isin(name_list)]
print(k_review.info())
k_review["name"].nunique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3015 entries, 0 to 7085
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   platform     3015 non-null   object
 1   name         3015 non-null   object
 2   rating       3015 non-null   int64 
 3   rating_date  3015 non-null   object
 4   comment      3015 non-null   object
dtypes: int64(1), object(4)
memory usage: 141.3+ KB
None


182

In [14]:
ss_review = ss_review[ss_review["name"].isin(name_list)]
print(ss_review.info())
ss_review["name"].nunique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1197 entries, 4 to 2041
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   platform  1197 non-null   object
 1   name      1197 non-null   object
 2   rating    1197 non-null   object
 3   comment   1197 non-null   object
dtypes: object(4)
memory usage: 46.8+ KB
None


54

In [15]:
m_review = m_review[m_review["name"].isin(name_list)]
print(m_review.info())
m_review["name"].nunique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 910 entries, 0 to 2922
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         910 non-null    object
 1   rating_date  910 non-null    object
 2   comment      910 non-null    object
 3   rating       910 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 35.5+ KB
None


62

In [16]:
n_review = n_review[n_review["name"].isin(name_list)]
print(n_review.info())
n_review["name"].nunique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13203 entries, 51 to 20972
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         13203 non-null  object 
 1   rating_date  13203 non-null  object 
 2   comment      12946 non-null  object 
 3   rating       13203 non-null  float64
dtypes: float64(1), object(3)
memory usage: 515.7+ KB
None


76

# 카테고리 분류하기

In [17]:
k_data_index_name = k_data.set_index("name")
all_count["category"] = k_data_index_name["category"]
all_count.head(20)

,count,category
name,,
1260이베리코흑돼지 강남직영점,1,"육류,고기"
24시와구아구,1,아구
60년전통 신촌황소곱창 선릉직영점,1,"곱창,막창"
7%칠백식당 포스코직영점,1,"육류,고기"
705사케집 선릉점,1,일본식주점
88노가리 선릉본점,1,"호프,요리주점"
CSI VR방탈출카페 대치선릉점,1,테마카페
JVL부대찌개 포스코점,1,"찌개,전골"
가나돈까스의집,2,"돈까스,우동"


In [26]:
all_cat_count = pd.DataFrame(all_count.reset_index().groupby("category").size())
all_cat_count.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76 entries, 갈비 to 회
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       76 non-null     int64
dtypes: int64(1)
memory usage: 1.2+ KB


# 메뉴 종류 파악해보기

In [ ]:
os.chdir('../kakao')
menus = pd.read_csv("kakao_restaurant_menu.csv", header=0, names=["name","menu"], usecols=[1, 2])

os.chdir("../data_analysis")
menus = menus[menus["name"].isin(name_list)]

# menu 누락 식당 - ['고운님한우곱창', '등촌샤브칼국수 선릉점', '명랑회관 선릉점','본죽&비빔밥cafe 선릉점', '쇼유식당',  '예향', '오또상스시', '원조광양불고기 삼성점', '유하제과', '코르바니', '하남돼지집 선릉1호점', ]
print(sorted(list(menus["name"].unique())))

In [ ]:
menus.info()

In [ ]:
menus["menu"] = menus.menu.str.split("(").str[0].str.strip(" ")
menus.drop_duplicates(inplace=True)
print(menus["menu"].nunique())
menus.info()

In [ ]:
print(menus["menu"].nunique())

In [ ]:
menus.value_counts()

In [ ]:
from collections import Counter
from wordcloud import WordCloud,STOPWORDS
from konlpy.tag import Okt
import matplotlib.pyplot as plt

%matplotlib inline
okt = Okt()

In [ ]:
words = okt.pos('한국어분석 스타트해보실까나?')
cnt = Counter(words)
cnt

In [ ]:
wc = WordCloud(background_color="#ffffff", width=1000, height=1000, max_font_size=300, max_words=1000, font_path="HangeulNuriR.ttf")

In [ ]:
test = wc.generate_from_text(str(n_review["comment"]))
plt.imshow(test)
plt.axis("off")
plt.show()

In [ ]:
# wc.to_file("test.png")